In [1]:
import numpy as np
from abc import ABC, abstractmethod
from progressbar import ProgressBar

# The goal of this notebook
Here are my solutions for the Optiver challenge regarding the amount of time it would take an ant to encounter food in various scenarios. Specifically, the question states:

>An ant leaves its anthill in order to forage for food. It moves with the speed of 10 cm per second, but it doesn't know where to go, therefore every second it moves randomly 10 cm directly north, south, east or west with equal probability.

To solve the various subquestions, we first define a `base_ant_random_walk` class, which we will use as a foundation for the various wandering ant scenarios (such as encoding the wandering behavior).


In [2]:
class base_ant_random_walk(ABC):
    def __init__(self, ant_number=100000, start_x=0.0, start_y=0.0, dt=1, v0=10):
        self.ant_number = ant_number                           # Number of ants we release
        self.start_x, self.start_y = start_x, start_y          # starting x and y coordinate
        self.xy_positions = self.initialize_position_array()   # initialized array of particle positions
        self.time = self.initialize_time_array()               # initialized array of particle time
        self.dt = dt                                           # integration timestep (seconds)
        self.v0 = v0                                           # walk speek (cm / s)
        
    def initialize_position_array(self):
        # Initializing the array in which we store the ants' x and y positions
        xy_positions = np.zeros(shape=(self.ant_number, 2))
        xy_positions[:, 0] = self.start_x
        xy_positions[:, 1] = self.start_y
        
        return xy_positions
    
    def initialize_time_array(self):
        return np.zeros(shape=(self.ant_number))
        
    def random_walk(self, xy_positions):
        # Generate an array with random integers between 0 - 3 which will set the direction of the random walks
        walk_direction = np.random.randint(low=0, high=4, size=self.ant_number)
        
        # If walk_direction == 0, move north by v0 * dt
        north = np.where(walk_direction == 0)[0]
        xy_positions[north, 1] += self.dt * self.v0
        # if walk_direction == 1, move south by v0 * dt
        south = np.where(walk_direction == 1)[0]
        xy_positions[south, 1] -= self.dt * self.v0
        # if walk_direction == 2, move east by v0 * dt
        east = np.where(walk_direction == 2)[0]
        xy_positions[east, 0] += self.dt * self.v0
        # if walk_direction == 3, move west by v0 * dt
        west = np.where(walk_direction == 3)[0]
        xy_positions[west, 0] -= self.dt * self.v0
        
        return xy_positions
    
    def calculate_walk(self, steps):
        for i in ProgressBar()(range(steps)):
            # Calculate the random walk procedure
            self.xy_positions = self.random_walk(self.xy_positions)
            # Update the time tracker of each ant
            self.time[~np.isnan(self.xy_positions[:, 0])] += 1
            # Set to np.nan all particles that are at or have crossed the boundary condition
            at_boundary = self.boundary_condition()
            self.xy_positions[at_boundary, :] = np.nan
            
        return self.xy_positions, self.time

    @abstractmethod
    def boundary_condition(self):
        # Define the specific condition when the ant encounters food
        pass
    
    def mean_travel_time(self, steps=1000):
        self.xy_positions, self.time = self.calculate_walk(steps=steps)
        # Determine the particles that are at the food
        at_food = np.isnan(self.xy_positions[:, 0])
        # Calculate the mean time
        mean_time = self.time[at_food].mean()
        std_time = self.time[at_food].std()
        error_time = std_time / np.sqrt(np.sum(at_food))
        
        # Calculate the number of ants that have reached the food
        at_food_percentage = np.sum(at_food) / self.ant_number * 100
        
        str_format = steps, mean_time, error_time
        print('After {} seconds, it takes the ant {:.2f}±{:.2f} seconds to encounter food.'.format(*str_format))
        print('In this time, {:.2f}% of ants have reached the food.\n'.format(at_food_percentage))  

# Question 1
The first question states:
> If the food is located on east-west lines 20 cm to the north and 20 cm to the south, as well as on north-south lines 20 cm to the east and 20 cm to the west from the anthill, how long will it take the ant to reach it on average?

We have already prescribed the random walk behavior in `base_ant_random_walk`, so to solve this question numerically we just need to specify the boundary condition. If $|x| \geq 20$ or $|y| \geq20$, the ant will have reached or crossed the boundaries, and therefore reached the food.

In [3]:
class ant_random_walk_question_1(base_ant_random_walk):
    def __init__(self):
        super().__init__()
        
    def boundary_condition(self):
        # if either the absolute x or y position is >= 20, then the ant has reached food
        boundary = (np.abs(self.xy_positions[:, 0]) >= 20) | (np.abs(self.xy_positions[:, 1]) >= 20)
        return boundary
        
ant_random_walk_question_1().mean_travel_time()

100% (1000 of 1000) |####################| Elapsed Time: 0:00:02 Time:  0:00:02


After 1000 seconds, it takes the ant 4.50±0.01 seconds to encounter food.
In this time, 100.00% of ants have reached the food.



# Question 2
The second question states:
> What is the average time the ant will reach food if it is located only on a diagonal line passing through (10cm, 0cm) and (0cm, 10cm) points?

The boundary condition for this question is that the food is found at `x + y = 10`, which we can easily encode in the `boundary_condition` function.

In [4]:
class ant_random_walk_question_2(base_ant_random_walk):
    def __init__(self):
        super().__init__()
        
    def boundary_condition(self):
        boundary = np.nansum(self.xy_positions, axis=1) == 10
        return boundary
        
ant_random_walk_question_2().mean_travel_time(steps=100)
ant_random_walk_question_2().mean_travel_time(steps=1000)
ant_random_walk_question_2().mean_travel_time(steps=10000)

100% (100 of 100) |######################| Elapsed Time: 0:00:00 Time:  0:00:00
  3% (38 of 1000) |                      | Elapsed Time: 0:00:00 ETA:   0:00:05

After 100 seconds, it takes the ant 7.69±0.05 seconds to encounter food.
In this time, 92.14% of ants have reached the food.



100% (1000 of 1000) |####################| Elapsed Time: 0:00:03 Time:  0:00:03
  0% (22 of 10000) |                     | Elapsed Time: 0:00:00 ETA:   0:00:46

After 1000 seconds, it takes the ant 24.89±0.29 seconds to encounter food.
In this time, 97.43% of ants have reached the food.



100% (10000 of 10000) |##################| Elapsed Time: 0:00:36 Time:  0:00:36


After 10000 seconds, it takes the ant 76.34±1.57 seconds to encounter food.
In this time, 99.23% of ants have reached the food.



Since we are not dealing with a closed boundary, the average time for the ant to reach the food is infinite. We can see this from the mean values if we give the ants more time to reach the food, as these don't converge to a single value (with the standard error also constantly increasing). However, we can get an indication of how likely an ant is to find the food within a certain time period. For example, within a time period of 10000 seconds (close to 3 hours), the ant has a 99.17% chance of encountaring food, taking on average 78.17±1.58 seconds. Depending on what question you are trying to answer about the ant, this can still be useful information.

# Question 3
The final question comes in two parts, where the first asks:
> Can you write a program that comes up with an estimate of average time to find food for any closed boundary around the anthill? 

In the current form of the code this is already achieved, as in principle any closed boundary can be specified around the anthill through the `boundary_condition` function in `base_ant_random_walk`. Therefore, we shall now proceed to the second part, which asks:

>What would be the answer if food is located outside an defined by ( (x – 2.5cm) / 30cm )2 + ( (y – 2.5cm) / 40cm )2 < 1 in coordinate system where the anthill is located at (x = 0cm, y = 0cm)? Provide us with a solution rounded to the nearest integer.

The notation is not entirely clear, but we will assume that the boundary indicates a elipse centered at (0, 0) following the following equation:

$\big((x - 2.5) / 30\big)^2 + \big((x - 2.5) / 40\big)^2 < 1$


In [5]:
class ant_random_walk_question_3(base_ant_random_walk):
    def __init__(self):
        super().__init__()
        
    def boundary_condition(self):
        boundary = np.square((self.xy_positions[:, 0] - 2.5) / 30) + np.square((self.xy_positions[:, 1] - 2.5) / 40) >= 1
        return boundary
        
ant_random_walk_question_3().mean_travel_time()

100% (1000 of 1000) |####################| Elapsed Time: 0:00:02 Time:  0:00:02


After 1000 seconds, it takes the ant 13.99±0.03 seconds to encounter food.
In this time, 100.00% of ants have reached the food.



As shown above, it takes an ant around 14 seconds to reach food within this closed boundary, and we can easily calculate this for any other closed boundary. As was shown for question 2, the code also works for an open boundary, but then instead of giving the mean time for the ant to reach the food, it can give the probability the ant will reach food within a given time period.